# Corpus

In [622]:
import pandas as pd                                     
import numpy as np                                      
import os 



import matplotlib.pyplot as plt

from datetime import datetime

%matplotlib inline
import seaborn as sns                                   # For pretty plots
import gensim
import nltk


from os import path
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from PIL import Image
from gensim import corpora, models, similarities
from nltk.corpus import stopwords

In [623]:
emails = pd.read_csv("hillary-clinton-emails/Emails.csv")
#emails.head()

In [624]:
emails_text = emails['ExtractedBodyText'].dropna()
#emails_text = emails_text.head(100)
#emails_text.shape
type(emails_text)

pandas.core.series.Series

In [625]:
emails_text = emails_text.apply(lambda x: x.lower())

In [626]:
emails_text = emails_text.apply(lambda x: x\
                                .replace(':',' ')\
                                .replace('—',' ')\
                                .replace('-',' ')\
                                .replace('.',' ')\
                                .replace(',',' ')\
                                .replace('.',' ')\
                                .replace('<',' ')\
                                .replace('>',' ')\
                                .replace('=',' ')\
                                .replace('•',' ')\
                                .replace("\\",' ')\
                                .replace('\n', ' ')\
                                .replace('^',' ')\
                                .replace('\\',' ')\
                                .replace('?',' ')\
                                .replace('\'',' ')\
                               )
#emails_text = emails_text.apply(lambda x: x.replace('—',' '))


In [627]:
type(emails_text)
#emails_text.iloc[0]
#emails_text

pandas.core.series.Series

In [628]:
#emails_text.head

In [629]:
documents = emails_text.tolist()
#documents

In [630]:
type(documents)
#documents

list

In [631]:
len(documents)

6742

In [632]:
stoplist = set(STOPWORDS)
stoplist.add('pls')
stoplist.add('yes')
stoplist.add('call')
stoplist.add('pm')
stoplist.add('no')
stoplist.add('com')
stoplist.add('doc')
stoplist.add('docx')
stoplist.add('pdf')
stoplist.add('mr')
stoplist.add('mrs')
stoplist.add('call')
stoplist.add('need')
stoplist.add('one')
stoplist.add('two')
stoplist.add('fyi')
stoplist.add('00')
stoplist.add('will')
stoplist.add('know')
stoplist.add('re')
stoplist.add('ok')
stoplist.add('also')
stoplist.add('see')
stoplist.add('us')
stoplist.add('good')
stoplist.add('thx')
stoplist.add('new')
stoplist.add('go')
stoplist.add('you')


for i in range(10,100) :
    stoplist.add(i)
print(stoplist)

{'are', 'itself', '', "she'll", 'two', 10, 11, 'most', "couldn't", 'an', 'after', 'its', 'been', 'ours', 'yourself', 12, 13, 14, 15, 16, 17, "don't", "you're", 'in', 20, 21, 22, 23, 24, 25, 26, 27, 28, 'would', 30, "he'll", 32, 33, 34, 35, 36, 37, 38, 39, 40, 'there', 'ok', 'thx', "she's", 45, 46, 47, 48, 49, 50, "he's", 'herself', "they'd", 'was', 55, 56, 57, "you'd", 59, 'docx', 61, 62, 63, 'both', 'could', 'no', 67, "mustn't", 69, 70, 71, 72, 73, 74, 'i', 'did', 'see', 'over', "aren't", "shouldn't", 'know', 18, 83, 84, 'doc', 'one', 19, 88, 89, 'ourselves', 91, 'such', 93, 94, 95, 96, "haven't", 98, 99, 'is', 'this', "shan't", "we're", "he'd", 'http', 'before', "it's", 'nor', 'our', 'very', 'theirs', 'pm', "that's", 'does', 'any', 'get', 29, "can't", 'doing', 'the', "you've", "we'd", 'and', "hasn't", 'hers', 'so', 31, "why's", 'good', 'have', 'off', 'them', 'having', 'again', 'also', "i'd", 'yours', 're', "we'll", 'how', 'cannot', 'during', "you'll", 'yourselves', 'new', 'other', "w

In [633]:
# remove list contains on stoplist and tokenize
texts = [[word for word in document.lower().split() if word not in stoplist and len(word) > 1]
         for document in documents]

In [634]:
len(texts)

6742

In [635]:
# counts the number of appartion of a word
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [636]:
len(frequency)

34511

In [637]:
# remove word that apprears only once
texts = [[token for token in text if frequency[token] > 1]
      for text in texts]

In [638]:
len(texts)

6742

In [639]:
#texts

In [640]:
dictionary = corpora.Dictionary(texts)
dictionary.save("part3/deerwester-email.dict")

In [641]:
print(dictionary)

Dictionary(17672 unique tokens: ['145', 'pouch', 'capabilities', 'rooms', 'mirage']...)


In [642]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [643]:
#print(corpus)

In [644]:
corpora.MmCorpus.serialize("part3/deerwester-email.mm", corpus)

In [645]:
dictionary = corpora.Dictionary.load("part3/deerwester-email.dict")


In [646]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=50)

In [647]:
model.print_topics(5)

[(39,
  '0.015*"received" + 0.014*"wjc" + 0.012*"hour" + 0.011*"mail" + 0.011*"message" + 0.011*"address" + 0.009*"state" + 0.009*"notetaker" + 0.008*"plane" + 0.008*"try"'),
 (21,
  '0.026*"israel" + 0.018*"settlements" + 0.011*"mcchrystal" + 0.009*"now" + 0.008*"israeli" + 0.008*"settlement" + 0.006*"bibi" + 0.006*"jewish" + 0.006*"jerusalem" + 0.006*"obama"'),
 (40,
  '0.028*"february" + 0.012*"2010" + 0.010*"anywhere" + 0.009*"boy" + 0.008*"san" + 0.007*"memorial" + 0.007*"stopping" + 0.007*"narrative" + 0.006*"card" + 0.006*"unless"'),
 (14,
  '0.019*"israeli" + 0.013*"conflict" + 0.013*"palestinian" + 0.007*"american" + 0.006*"army" + 0.006*"tea" + 0.006*"netanyahu" + 0.006*"evidence" + 0.006*"security" + 0.005*"russia"'),
 (36,
  '0.018*"beck" + 0.010*"modern" + 0.007*"arizona" + 0.006*"dinner" + 0.006*"cheryl" + 0.006*"right" + 0.006*"interested" + 0.006*"happening" + 0.006*"answers" + 0.006*"leaks"')]

In [648]:
print(model)

LdaModel(num_terms=17672, num_topics=50, decay=0.5, chunksize=2000)


In [649]:
#print(corpus)